In [1]:
import numpy as np
from PIL import Image
import hashlib

class BinaryToImageDiffusion:
    def __init__(self, seed=42):
        """
        Initialize binary-to-image generator using stable diffusion principles.

        Args:
            seed: Base random seed for reproducibility
        """
        self.seed = seed

    def _binary_to_seed(self, binary_data):
        """
        Convert binary data to a unique seed for image generation.

        Args:
            binary_data: Binary string

        Returns:
            Integer seed derived from binary data
        """
        # Use hash to create reproducible seed from binary
        hash_obj = hashlib.sha256(binary_data.encode())
        seed = int(hash_obj.hexdigest(), 16) % (2**32)
        return seed

    def _binary_to_latent_features(self, binary_data, num_features=128):
        """
        Convert binary data into latent space features.
        Each chunk of binary controls different aspects of the generation.

        Args:
            binary_data: Binary string
            num_features: Number of latent features to generate

        Returns:
            Array of latent features
        """
        # Pad binary to ensure we have enough data
        binary_len = len(binary_data)
        needed_bits = num_features * 8

        if binary_len < needed_bits:
            # Pad with pattern derived from original data
            pattern = binary_data if binary_data else "10101010"
            repeats = (needed_bits // len(pattern)) + 1
            binary_padded = (pattern * repeats)[:needed_bits]
        else:
            binary_padded = binary_data[:needed_bits]

        # Convert chunks to feature values
        features = []
        chunk_size = 8

        for i in range(0, len(binary_padded), chunk_size):
            chunk = binary_padded[i:i+chunk_size]
            if len(chunk) == chunk_size:
                # Convert binary chunk to float value [-1, 1]
                int_val = int(chunk, 2)
                normalized = (int_val / 127.5) - 1.0  # Map [0, 255] to [-1, 1]
                features.append(normalized)

        return np.array(features[:num_features])

    def _initialize_noise(self, width, height, channels, seed):
        """
        Initialize pure noise as starting point for diffusion.

        Args:
            width: Image width
            height: Image height
            channels: Number of color channels
            seed: Random seed

        Returns:
            Random noise array
        """
        np.random.seed(seed)
        noise = np.random.randn(height, width, channels)
        return noise

    def _apply_latent_conditioning(self, noise, latent_features, step, total_steps):
        """
        Apply latent features to guide the denoising process.
        This is where binary data influences image generation.

        Args:
            noise: Current noisy image
            latent_features: Features derived from binary data
            step: Current diffusion step
            total_steps: Total number of steps

        Returns:
            Conditioned noise
        """
        h, w, c = noise.shape

        # Create spatial conditioning maps from latent features
        # Different features control different spatial regions
        conditioned = noise.copy()

        # Divide features into groups for different purposes
        num_features = len(latent_features)

        # Color control (first 1/3 of features)
        color_features = latent_features[:num_features//3]
        for i, feat in enumerate(color_features):
            channel = i % c
            conditioned[:, :, channel] += feat * 0.5

        # Spatial structure control (middle 1/3)
        structure_features = latent_features[num_features//3:2*num_features//3]
        for i, feat in enumerate(structure_features):
            # Create gradients and patterns
            x = np.linspace(-1, 1, w)
            y = np.linspace(-1, 1, h)
            X, Y = np.meshgrid(x, y)

            pattern = np.sin(X * (i + 1) * np.pi) * np.cos(Y * (i + 1) * np.pi)
            pattern = pattern * feat * 0.3

            for ch in range(c):
                conditioned[:, :, ch] += pattern

        # Texture control (last 1/3)
        texture_features = latent_features[2*num_features//3:]
        for i, feat in enumerate(texture_features):
            # Add frequency-based patterns
            freq = (i + 1) * 2
            noise_texture = np.random.randn(h, w) * feat * 0.2
            for ch in range(c):
                conditioned[:, :, ch] += noise_texture

        return conditioned

    def _denoise_step(self, noisy_img, latent_features, step, total_steps):
        """
        Perform one denoising step guided by latent features.

        Args:
            noisy_img: Current noisy image
            latent_features: Binary-derived features
            step: Current step (counts down from total_steps to 0)
            total_steps: Total denoising steps

        Returns:
            Less noisy image
        """
        # Calculate denoising strength (stronger as we progress)
        progress = (total_steps - step) / total_steps
        denoise_strength = progress ** 0.5

        # Apply latent conditioning
        conditioned = self._apply_latent_conditioning(
            noisy_img, latent_features, step, total_steps
        )

        # Denoise: gradually reduce random variation
        # Keep some of the conditioned signal, reduce pure noise
        denoised = conditioned * (1 - denoise_strength * 0.7)

        # Add decreasing amount of fresh noise (like DDPM)
        if step > 0:
            noise_scale = (step / total_steps) * 0.3
            fresh_noise = np.random.randn(*noisy_img.shape) * noise_scale
            denoised = denoised + fresh_noise

        return denoised

    def _post_process(self, img_array):
        """
        Convert generated array to viewable image.

        Args:
            img_array: Raw generated array

        Returns:
            Processed uint8 image array
        """
        # Normalize to [0, 255]
        img_min = img_array.min()
        img_max = img_array.max()

        if img_max > img_min:
            normalized = (img_array - img_min) / (img_max - img_min)
        else:
            normalized = np.zeros_like(img_array)

        img_uint8 = (normalized * 255).astype(np.uint8)

        return img_uint8

    def generate_image_from_binary(self, binary_data, output_path,
                                   width=512, height=512,
                                   diffusion_steps=50):
        """
        Generate an image from binary data using stable diffusion process.

        Args:
            binary_data: Binary string (e.g., "10110101...")
            output_path: Where to save generated image
            width: Image width
            height: Image height
            diffusion_steps: Number of denoising steps (more = higher quality)

        Returns:
            dict with generation info
        """
        print("=" * 70)
        print("GENERATING IMAGE FROM BINARY DATA")
        print("=" * 70)

        # Validate binary data
        if not all(bit in '01' for bit in binary_data):
            raise ValueError("Binary data must contain only '0' and '1' characters")

        print(f"\n📊 Input Information:")
        print(f"   Binary length: {len(binary_data)} bits")
        print(f"   Image size: {width}x{height}")
        print(f"   Diffusion steps: {diffusion_steps}")

        # Step 1: Convert binary to seed and latent features
        print(f"\n🔄 Step 1: Converting binary to latent features...")
        seed = self._binary_to_seed(binary_data)
        print(f"   Generated seed: {seed}")

        latent_features = self._binary_to_latent_features(binary_data, num_features=128)
        print(f"   Latent features: {len(latent_features)} dimensions")
        print(f"   Feature range: [{latent_features.min():.3f}, {latent_features.max():.3f}]")

        # Step 2: Initialize pure noise
        print(f"\n🎲 Step 2: Initializing random noise...")
        img = self._initialize_noise(width, height, 3, seed)
        print(f"   Noise shape: {img.shape}")

        # Step 3: Iterative denoising (reverse diffusion)
        print(f"\n🎨 Step 3: Denoising process (generating image)...")

        for step in range(diffusion_steps, 0, -1):
            img = self._denoise_step(img, latent_features, step, diffusion_steps)

            # Progress updates
            if step % 10 == 0 or step == 1:
                progress = ((diffusion_steps - step) / diffusion_steps) * 100
                print(f"   Progress: {progress:.0f}% (step {diffusion_steps - step}/{diffusion_steps})")

        # Step 4: Post-process and save
        print(f"\n💾 Step 4: Post-processing and saving...")
        img_uint8 = self._post_process(img)

        output = Image.fromarray(img_uint8)
        output.save(output_path)

        print(f"   ✓ Image saved: {output_path}")

        # Calculate statistics
        unique_colors = len(np.unique(img_uint8.reshape(-1, 3), axis=0))

        print(f"\n✅ Generation complete!")

        return {
            'binary_length': len(binary_data),
            'seed_used': seed,
            'image_size': f"{width}x{height}",
            'diffusion_steps': diffusion_steps,
            'unique_colors': unique_colors,
            'output_path': output_path
        }

    def extract_binary_from_image(self, image_path, expected_length):
        """
        Extract the approximate binary data that generated an image.
        Note: This is an approximation and may not be exact due to the
        generative nature of the process.

        Args:
            image_path: Path to generated image
            expected_length: Expected length of binary data

        Returns:
            Approximated binary string
        """
        print("\n" + "=" * 70)
        print("EXTRACTING BINARY SIGNATURE FROM IMAGE")
        print("=" * 70)

        # Load image
        img = Image.open(image_path)
        img_array = np.array(img)

        print(f"\n📊 Image loaded: {img_array.shape}")

        # Extract features from image
        h, w, c = img_array.shape

        # Sample key pixels and convert to binary
        binary_approximation = ""

        # Sample from different regions
        samples_needed = (expected_length + 7) // 8

        for i in range(samples_needed):
            # Get pixel from pseudo-random location based on index
            y = (i * 31) % h
            x = (i * 47) % w

            # Get average color value
            pixel_val = int(np.mean(img_array[y, x]))

            # Convert to binary
            binary_chunk = format(pixel_val, '08b')
            binary_approximation += binary_chunk

        # Trim to expected length
        binary_approximation = binary_approximation[:expected_length]

        print(f"✓ Extracted {len(binary_approximation)} bits")

        return binary_approximation

    def text_to_binary(self, text):
        """Convert text to binary string."""
        return ''.join(format(ord(char), '08b') for char in text)

    def binary_to_text(self, binary):
        """Convert binary string to text."""
        chars = [binary[i:i+8] for i in range(0, len(binary), 8)]
        return ''.join(chr(int(char, 2)) for char in chars if len(char) == 8)


# Example usage
if __name__ == "__main__":
    generator = BinaryToImageDiffusion(seed=42)

    # Example 1: Generate image from binary data
    print("\n" + "🎯 " * 20)
    print("EXAMPLE 1: GENERATING IMAGE FROM BINARY DATA")
    print("🎯 " * 20)

    binary_data = "11001100101010110011110000111100110011001100110011001100"

    result = generator.generate_image_from_binary(
        binary_data=binary_data,
        output_path='generated_from_binary.png',
        width=400,
        height=400,
        diffusion_steps=40
    )

    print("\n📋 Generation Summary:")
    for key, value in result.items():
        print(f"   {key}: {value}")

    # Example 2: Generate image from text message
    print("\n" + "🎯 " * 20)
    print("EXAMPLE 2: GENERATING IMAGE FROM TEXT MESSAGE")
    print("🎯 " * 20)

    message = "Hello AI World 2024!"
    binary_message = generator.text_to_binary(message)

    print(f"\n💬 Message: '{message}'")
    print(f"📝 Binary: {binary_message[:50]}... ({len(binary_message)} bits)")

    result = generator.generate_image_from_binary(
        binary_data=binary_message,
        output_path='generated_from_text.png',
        width=400,
        height=400,
        diffusion_steps=40
    )

    print("\n📋 Generation Summary:")
    for key, value in result.items():
        print(f"   {key}: {value}")

    # Example 3: Generate multiple images from different binary patterns
    print("\n" + "🎯 " * 20)
    print("EXAMPLE 3: GENERATING MULTIPLE IMAGES")
    print("🎯 " * 20)

    patterns = {
        'pattern_1': "11111111" * 10,  # All ones
        'pattern_2': "00000000" * 10,  # All zeros
        'pattern_3': "10101010" * 10,  # Alternating
        'pattern_4': "11001100" * 10,  # Double alternating
    }

    for name, pattern in patterns.items():
        print(f"\n🎨 Generating {name}...")
        result = generator.generate_image_from_binary(
            binary_data=pattern,
            output_path=f'{name}.png',
            width=300,
            height=300,
            diffusion_steps=30
        )
        print(f"   ✓ Saved as {result['output_path']}")

    print("\n" + "=" * 70)
    print("🎉 ALL EXAMPLES COMPLETED SUCCESSFULLY!")
    print("=" * 70)
    print("\n💡 Tip: Different binary inputs create completely different images!")
    print("💡 Try changing the binary data to see how it affects generation!")


🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 
EXAMPLE 1: GENERATING IMAGE FROM BINARY DATA
🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 
GENERATING IMAGE FROM BINARY DATA

📊 Input Information:
   Binary length: 56 bits
   Image size: 400x400
   Diffusion steps: 40

🔄 Step 1: Converting binary to latent features...
   Generated seed: 491689491
   Latent features: 128 dimensions
   Feature range: [-0.529, 0.600]

🎲 Step 2: Initializing random noise...
   Noise shape: (400, 400, 3)

🎨 Step 3: Denoising process (generating image)...
   Progress: 0% (step 0/40)
   Progress: 25% (step 10/40)
   Progress: 50% (step 20/40)
   Progress: 75% (step 30/40)
   Progress: 98% (step 39/40)

💾 Step 4: Post-processing and saving...
   ✓ Image saved: generated_from_binary.png

✅ Generation complete!

📋 Generation Summary:
   binary_length: 56
   seed_used: 491689491
   image_size: 400x400
   diffusion_steps: 40
   unique_colors: 4782
   output_path: generated_from_binary.png

🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯